In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [2]:
DIRECTORY = './data/'
categories = pd.read_csv(DIRECTORY + 'item_categories.csv')
items = pd.read_csv(DIRECTORY + 'items.csv')
shops = pd.read_csv(DIRECTORY + 'shops.csv')
train = pd.read_csv(DIRECTORY + 'sales_train.csv', parse_dates=['date'], date_format='%d.%m.%Y')
test = pd.read_csv(DIRECTORY + 'test.csv')
train['month'] = train['date'].dt.to_period('M')

In [6]:
combined = items.merge(categories, on=['item_category_id'], how='left')

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра)
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD


In [42]:
# Add in features that contribute to similarity

combined['item_first_word'] = combined['item_name'].str.split().str[0]
combined['item_second_word'] = combined['item_name'].str.split().str[:2].str.join(' ')
combined['item_parenthesis'] = combined['item_name'].str.extract(r'\((.*?)\)')
combined['item_bracket'] = combined['item_name'].str.extract(r'\[(.*?)\]')
combined['item_before_parenthesis'] = np.where(combined['item_parenthesis'].isna(), np.nan, items['item_name'].str.split('(').str[0].str.split().str[-1])
combined['item_before_bracket'] = np.where(combined['item_bracket'].isna(), np.nan, items['item_name'].str.split('[').str[0].str.split().str[-1])
combined['category_first_word'] = combined['item_category_name'].str.split().str[0]
combined['item_first_word'].value_counts()

item_first_word
Фигурка        599
Футболка       356
СБ.            333
Мягкая         302
V/A            286
              ... 
МУЗЫКАЛЬНЫЕ      1
АТЛАНТИДА        1
МУЛ              1
МУЛАН/МУЛАН      1
!                1
Name: count, Length: 5252, dtype: int64

In [43]:
factorized = combined.copy()
for col in combined.columns:
    if col in ['item_name', 'item_id']: continue
    factorized[col], _ = pd.factorize(factorized[col])
factorized.head()

,item_name,item_id,item_category_id,item_category_name,item_first_word,item_second_word,item_parenthesis,item_bracket,item_before_bracket,item_before_parenthesis,category_first_word
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,0,0,0,0,0,-1,-1,0,0
1,!ABBYY FineReader 12 Professional Edition Full...,1,1,1,1,1,-1,0,0,-1,1
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,0,0,2,2,1,-1,-1,1,0
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,0,0,3,3,2,-1,-1,2,0
4,***КОРОБКА (СТЕКЛО) D,4,0,0,4,4,3,-1,-1,3,0


In [48]:
# Compare to each other
small_group = factorized[factorized['item_first_word'] == 124]
cross = pd.merge(left=small_group, right=small_group, how='cross')
columns = factorized.columns